# Test notebook for structure checking

A complete checking analysis of a single structure follows.

#### Setting python paths

#### Basic imports

In [1]:
import biobb_structure_checking
import biobb_structure_checking.constants as cts
from biobb_structure_checking.structure_checking import StructureChecking
base_dir_path=biobb_structure_checking.__path__[0]


Setting default data and paths

In [2]:
args = cts.set_defaults(base_dir_path,{'notebook':True})

General help

In [3]:
with open(args['commands_help_path']) as help_file:
    print(help_file.read())


BioBB's check_structure.py performs MDWeb structure checking set as a command line
utility.

commands:     Help on available commands
command_list: Run all tests from conf file
checkall:     Perform all checks without fixes
fixall:       Performs a default fix (v1.1)
load:         Stores structure on local cache and provide basic statistics

1. System Configuration
models [--select model_num]
    Detect/Select Models
chains [--select chain_ids]
    Detect/Select Chains
inscodes [--renum]
    Detects residues with insertion codes. --renum (v1.1) renumbers residues
altloc [--select occupancy| alt_id | list of res_id:alt_id]
    Detect/Select Alternative Locations
metals [--remove All | None | Met_ids_list | Residue_list]
    Detect/Remove Metals
ligands [--remove All | None | Res_type_list | Residue_list]
    Detect/Remove Ligands
hetatm [--remove All | None | Res_type_list | Residue_list] (v1.1)
    Detect/Remove Ligands, revert modified residues
water [--remove Yes|No]
    Remove Wate

Set input (PDB or local file, pdb or mmCif formats allowed) and output (local file, pdb format).  
Use pdb:pdbid for downloading structure from PDB (RCSB)

In [4]:
args['input_structure_path'] ='pdb:2ki5'
args['output_structure_path']='temp.pdb'

Initializing checking engine, loading structure and showing statistics

In [5]:

st_c = StructureChecking(base_dir_path,args)


Structure exists: '/home/gelpi/miniconda3/envs/devChecking/lib/python3.6/site-packages/biobb_structure_checking/tmpPDB/ki/2ki5.cif' 
Structure pdb:2ki5 loaded
 PDB id: 2KI5
 Title: HERPES SIMPLEX TYPE-1 THYMIDINE KINASE IN COMPLEX WITH THE DRUG ACICLOVIR AT 1.9A RESOLUTION
 Experimental method: X-RAY DIFFRACTION
 Keywords: TRANSFERASE
 Resolution: 1.90 A

 Num. models: 1
 Num. chains: 2 (A: Protein, B: Protein)
 Num. residues:  908
 Num. residues with ins. codes:  0
 Num. HETATM residues:  296
 Num. ligands or modified residues:  4
 Num. water mol.:  292
 Num. atoms:  4961
Small mol ligands found
SO4 A4
AC2 A1
SO4 B3
AC2 B2



#### models
Checks for the presence of models in the structure. 
MD simulations require a single structure, although some structures (e.g. biounits) may be defined as a series of models, in such case all of them are usually required.  
Use models('N') to select model num N for further analysis

In [6]:
st_c.models()

Running models.
1 Model(s) detected
Single model found
Running  check_only. Nothing else to do.


#### chains
Checks for chains (also obtained from print_stats), and allow to select one or more.   
MD simulations are usually performed with complete structures. However input structure may contain several copies of the system, or contains additional chains like peptides or nucleic acids that may be removed. 
Use chains('X,Y') to select chain(s) X and Y to proceed

In [7]:
st_c.chains('A')


Running chains. Options: A 
2 Chain(s) detected
 A: Protein
 B: Protein
A
Selecting chain(s) A


#### altloc
Checks for the presence of residues with alternative locations. Atoms with alternative coordinates and their occupancy are reported.  
MD simulations requires a single position for each atom.  
Use altloc('occupancy | alt_ids | list of res:id) to select the alternative


In [8]:
st_c.altloc()

Running altloc.
Detected 2 residues with alternative location labels
AC2 A1
  C3'  A (1.00) B (0.01)
  O3'  A (1.00) B (0.01)
  C2'  A (1.00) B (0.01)
  O1'  A (1.00) B (0.01)
  C1'  A (1.00) B (0.01)
HOH A779
  O    A (1.00)
Running  check_only. Nothing else to do.


#### metals
Detects HETATM being metal ions allow to selectively remove them.  
To remove use metals (' All | None | metal_type list | residue list ')

In [9]:
st_c.metals()

Running metals.
No metal ions found
Running  check_only. Nothing else to do.


#### ligands
Detects HETATM (excluding Water molecules) to selectively remove them.  
To remove use ligands('All | None | Residue List (by id, by num)')


In [10]:
st_c.ligands()

Running ligands.
2 Ligands detected
 SO4 A4
 AC2 A1
Running  check_only. Nothing else to do.


#### rem_hydrogen
Detects and remove hydrogen atoms. 
MD setup can be done with the original H atoms, however to prevent from non standard labelling, remove them is safer.  
To remove use rem_hydrogen('yes')


In [11]:
st_c.rem_hydrogen()

Running rem_hydrogen.
No residues with Hydrogen atoms found
Running  check_only. Nothing else to do.


#### water
Detects water molecules and allows to remove them
Crystallographic water molecules may be relevant for keeping the structure, however in most cases only some of them are required. These can be later added using other methods (titration) or manually.

To remove water molecules use water('yes')


In [12]:
st_c.water()

Running water.
144 Water molecules detected
Running  check_only. Nothing else to do.


#### amide
Amide terminal atoms in Asn ang Gln residues can be labelled incorrectly.  
amide suggests possible fixes by checking the sourrounding environent.

To fix use amide ('All | None | residue_list')

Note that the inversion of amide atoms may trigger additional contacts. 

In [13]:
st_c.amide()

Running amide.
No unusual contact(s) involving amide atoms found
Running  check_only. Nothing else to do.


#### chiral
Side chains of Thr and Ile are chiral, incorrect atom labelling lead to the wrong chirality.  
To fix use chiral('All | None | residue_list')

In [14]:
st_c.chiral()

Running chiral.
No residues with incorrect side-chain chirality found
Running  check_only. Nothing else to do.


#### Backbone
Detects and fixes several problems with the backbone


In [20]:
st_c.backbone('--fix_chain all')

Running backbone. Options: --fix_chain all 
4 Residues with missing backbone atoms found
 VAL A70    CA,C,O,OXT
 GLY A148   CA,C,O,OXT
 GLY A264   CA,C,O,OXT
 ALA A375   CA,C,O,OXT
3 Backbone breaks found
 VAL A70    - ASP A77    
 GLY A148   - PRO A154   
 GLY A264   - PRO A280   
Main chain fixes

                         MODELLER 9.21, 2018/12/06, r11301

     PROTEIN STRUCTURE MODELLING BY SATISFACTION OF SPATIAL RESTRAINTS


                     Copyright(c) 1989-2018 Andrej Sali
                            All Rights Reserved

                             Written by A. Sali
                               with help from
              B. Webb, M.S. Madhusudhan, M-Y. Shen, G.Q. Dong,
          M.A. Marti-Renom, N. Eswar, F. Alber, M. Topf, B. Oliva,
             A. Fiser, R. Sanchez, B. Yerkovich, A. Badretdinov,
                     F. Melo, J.P. Overington, E. Feyfant
                 University of California, San Francisco, USA
                    Rockefeller University, New York

KeyError: 0

#### fixside
Detects and re-built missing protein side chains.   
To fix use fixside('All | None | residue_list')

In [16]:
st_c.fixside()

Running fixside.
3 Residues with missing side chain atoms found
 MET A46    CG,SD,CE
 PRO A154   CG,CD
 PRO A280   CG,CD
Running  check_only. Nothing else to do.


#### getss
Detects possible -S-S- bonds based on distance criteria.
Proper simulation requires those bonds to be correctly set.

In [17]:
st_c.getss()

Running getss.
No SS bonds detected
Running  check_only. Nothing else to do.


#### clashes
Detects steric clashes based on distance criteria.  
Contacts are classified in: 
* Severe: Too close atoms, usually indicating superimposed structures or badly modelled regions. Should be fixed.
* Apolar: Vdw colissions.Usually fixed during the simulation.
* Polar and ionic. Usually indicate wrong side chain conformations. Usually fixed during the simulation


In [18]:
st_c.clashes()

Running clashes.
No severe clashes detected
4 Steric apolar clashes detected
 GLY A61.O    THR A65.CG2     2.799 A
 MET A85.CG   GLU A374.OE1    2.857 A
 ARG A89.CD   GLU A95.OE1     2.776 A
 LEU A232.O   ARG A236.CG     2.741 A
1 Steric polar_acceptor clashes detected
 ASP A55.O    VAL A204.O      3.035 A
2 Steric polar_donor clashes detected
 LYS A62.NZ   ARG A163.NH1    2.578 A
 ARG A212.NH2 ARG A216.NH2    3.075 A
2 Steric positive clashes detected
 LYS A62.NZ   ARG A163.NH1    2.578 A
 ARG A212.NH2 ARG A216.NH2    3.075 A
No negative clashes detected
Running  check_only. Nothing else to do.
